In [1]:
# ===== Mozilla – Cell 1: imports, mount, paths =====

import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')

# Root where your v02 folder lives (same as Eclipse)
DATA_ROOT   = "/content/drive/MyDrive/msr2013-bug_dataset-master/data/v02"
MOZILLA_DIR = f"{DATA_ROOT}/mozilla"

ASSIGNED_FILE   = f"{MOZILLA_DIR}/assigned_to.json"
SHORT_DESC_FILE = f"{MOZILLA_DIR}/short_desc.json"
BUG_STATUS_FILE = f"{MOZILLA_DIR}/bug_status.json"
RESOLUTION_FILE = f"{MOZILLA_DIR}/resolution.json"

print("Mozilla folder:", MOZILLA_DIR)
print("Files:")
print("  ", ASSIGNED_FILE)
print("  ", SHORT_DESC_FILE)
print("  ", BUG_STATUS_FILE)
print("  ", RESOLUTION_FILE)


Mounted at /content/drive
Mozilla folder: /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla
Files:
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/assigned_to.json
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/short_desc.json
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/bug_status.json
   /content/drive/MyDrive/msr2013-bug_dataset-master/data/v02/mozilla/resolution.json


In [2]:
# ===== Mozilla – Cell 2: build classifier-like DataFrame from JSONs =====

import json
import pandas as pd

def load_last_event_table(path, root_key, out_col):
    """
    MSR2013 JSON format:
      { root_key : { bug_id : [ {when, what, who}, ... ], ... } }

    For each bug_id, take the LAST event (max 'when') and keep its 'what'.
    Returns a DataFrame with columns: ['bug_id', out_col].
    """
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    inner = raw[root_key]   # dict: bug_id -> list of events
    rows = []

    for bug_id, events in inner.items():
        if not events:
            continue
        last = max(events, key=lambda e: e.get("when", 0))
        rows.append({
            "bug_id": int(bug_id),
            out_col: str(last.get("what", ""))
        })

    return pd.DataFrame(rows)

# 2.1 flatten each JSON into (bug_id, value) tables
assigned_df   = load_last_event_table(ASSIGNED_FILE,   "assigned_to", "assigned_to")
short_df      = load_last_event_table(SHORT_DESC_FILE, "short_desc",  "short_desc")
bug_status_df = load_last_event_table(BUG_STATUS_FILE, "bug_status",  "bug_status")
resolution_df = load_last_event_table(RESOLUTION_FILE, "resolution",  "resolution")

print("assigned_df head:")
display(assigned_df.head())
print("short_df head:")
display(short_df.head())
print("bug_status_df head:")
display(bug_status_df.head())
print("resolution_df head:")
display(resolution_df.head())

# 2.2 merge on bug_id to get one row per bug
df = (assigned_df
      .merge(short_df,      on="bug_id", how="inner")
      .merge(bug_status_df, on="bug_id", how="inner")
      .merge(resolution_df, on="bug_id", how="inner"))

print("Merged rows before filters:", len(df))

# 2.3 keep only RESOLVED / VERIFIED / CLOSED **and** FIXED
status_upper = df["bug_status"].str.upper()
res_upper    = df["resolution"].str.upper()

mask_status = status_upper.isin(["RESOLVED", "VERIFIED", "CLOSED"])
mask_res    = res_upper.str.contains("FIXED", na=False)

df = df[mask_status & mask_res].copy()
print("Rows after resolved+fixed filter:", len(df))

# 2.4 classifier-style columns: owner, issue_title, description
df["owner"]       = df["assigned_to"].astype(str)
df["issue_title"] = df["short_desc"].astype(str)
df["description"] = ""   # v02 has only short description

# keep only developers with ≥ MIN_BUGS (same spirit as Eclipse setup)
MIN_BUGS = 2
owner_counts = df["owner"].value_counts()
keep_owners  = owner_counts[owner_counts >= MIN_BUGS].index
df = df[df["owner"].isin(keep_owners)].reset_index(drop=True)

print("Rows after MIN_BUGS filter:", len(df))
print("Number of developers with ≥2 bugs:", df["owner"].nunique())

# 2.5 encode dev_id and build classifier-like DataFrame
df["dev_id"] = df["owner"].astype("category").cat.codes
n_devs = df["dev_id"].nunique()
print("Final number of developers:", n_devs)

clf_df = df[["owner", "issue_title", "description", "dev_id"]]
display(clf_df.head())


assigned_df head:


bug_id           assigned_to
0     538     terry@mozilla.org
1      91    rickg@netscape.com
2     104  brendan@netscape.com
3     133    ftang@netscape.com
4     174    gagan@netscape.com

short_df head:


bug_id                                         short_desc
0     827            Crashed after adding comments to a bug.
1     818  Internet Explorer can't handle server-push pages.
2     858   [Feature] JavaScript auto-disable per-domain RFE
3     785             {sink} Handling frameset's with bodies
4     672                               yahoo margin problem

bug_status_df head:


bug_id bug_status
0     539   VERIFIED
1     538   VERIFIED
2     540        NEW
3     221   VERIFIED
4     360   VERIFIED

resolution_df head:


bug_id resolution
0     539      FIXED
1     538      FIXED
2     221    INVALID
3     360    INVALID
4     381    INVALID

Merged rows before filters: 394878
Rows after resolved+fixed filter: 115540
Rows after MIN_BUGS filter: 114493
Number of developers with ≥2 bugs: 1843
Final number of developers: 1843


owner                                        issue_title  \
0   terry@mozilla.org   Bugzilla can't search bug lists for a text match   
1  rickg@netscape.com               document properties cannot be listed   
2  ftang@netscape.com  Navigator draws entities like &lt;&amp;&gt; as...   
3  gagan@netscape.com  URL: mailto:'s -> Failed assert on exit at mkg...   
4  ftang@netscape.com  Incorrect display of CP-1250 pages in Unix ver...   

  description  dev_id  
0                1672  
1                1451  
2                 656  
3                 662  
4                 656

In [3]:
# ===== Mozilla – Cell 3: tokenisation + train/val/test splits =====

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

MAX_VOCAB  = 50000
MAX_LEN    = 300
BATCH_SIZE = 256
VAL_SIZE   = 0.1   # 10% of train_full used as validation

# 3.1 text = title + description
clf_df["text"] = (
    clf_df["issue_title"].fillna("").astype(str)
    + " "
    + clf_df["description"].fillna("").astype(str)
)

# 3.2 labels
y_all = clf_df["dev_id"].astype("int32").values
n_devs = clf_df["dev_id"].nunique()
print("Number of developers:", n_devs)

texts = clf_df["text"].tolist()

# 3.3 Keras tokenizer on all Mozilla text
tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
seqs = tokenizer.texts_to_sequences(texts)

X_all = pad_sequences(
    seqs,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)
print("X_all shape:", X_all.shape)

# 3.4 stratified 80/20 train+test split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=y_all,
)

# 3.5 validation split from train_full
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full,
    y_train_full,
    test_size=VAL_SIZE,
    random_state=42,
    stratify=y_train_full,
)

print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)

# 3.6 vocab size actually used
vocab_size = min(MAX_VOCAB, len(tokenizer.word_index) + 1)
print("Vocab size (capped):", vocab_size)


/tmp/ipython-input-3773736186.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clf_df["text"] = (


Number of developers: 1843
X_all shape: (114493, 300)
Train: (82434, 300)  Val: (9160, 300)  Test: (22899, 300)
Vocab size (capped): 49644


In [4]:
# ===== Mozilla – Cell 4: DBRNN model (BiLSTM + attention) =====

import tensorflow as tf
from tensorflow.keras import layers, models

EMBED_DIM  = 128
LSTM_UNITS = 128

def build_dbrnn_model(vocab_size, max_len, n_classes):
    inputs = layers.Input(shape=(max_len,), dtype="int32")

    x = layers.Embedding(
        input_dim=vocab_size,
        output_dim=EMBED_DIM,
        mask_zero=False
    )(inputs)

    x = layers.Bidirectional(
        layers.LSTM(LSTM_UNITS, return_sequences=True)
    )(x)   # (B, T, 2*LSTM_UNITS)

    # simple additive attention
    e = layers.Dense(1, activation="tanh")(x)           # (B, T, 1)

    alpha = layers.Lambda(
        lambda t: tf.nn.softmax(t, axis=1),
        name="attn_weights"
    )(e)                                                # (B, T, 1)

    def apply_attention(tensors):
        h, a = tensors   # h: (B, T, 2H), a: (B, T, 1)
        return tf.reduce_sum(h * a, axis=1)   # (B, 2H)

    doc_repr = layers.Lambda(apply_attention, name="doc_repr")([x, alpha])

    outputs = layers.Dense(n_classes, activation="softmax")(doc_repr)

    model = models.Model(inputs=inputs, outputs=outputs, name="DBRNN_Mozilla")
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

n_classes = n_devs
model = build_dbrnn_model(vocab_size, MAX_LEN, n_classes)
model.summary()


Model: "DBRNN_Mozilla"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 300, 128)  │  6,354,432 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 300, 256)  │    263,168 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 300, 1)    │        257 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attn_weights        │ (None, 300, 1)    │          0 │ dense[0][0]       │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ doc_repr (Lambda)   │ (None, 256)       │          0 │ bidirectional[0]… │
│                     │                   │            │ attn_weights[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1843)      │    473,651 │ doc_repr[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,091,508 (27.05 MB)

 Trainable params: 7,091,508 (27.05 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# ===== Mozilla – Cell 5: train DBRNN =====

EPOCHS = 10   # with early stopping

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    callbacks=callbacks,
)


Epoch 1/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 33s 85ms/step - accuracy: 0.3831 - loss: 4.8439 - val_accuracy: 0.3920 - val_loss: 4.4366
Epoch 2/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 37s 85ms/step - accuracy: 0.3939 - loss: 4.4550 - val_accuracy: 0.3920 - val_loss: 4.3838
Epoch 3/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - accuracy: 0.3925 - loss: 4.3667 - val_accuracy: 0.3920 - val_loss: 4.2886
Epoch 4/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 28s 87ms/step - accuracy: 0.3923 - loss: 4.1594 - val_accuracy: 0.3927 - val_loss: 4.2166
Epoch 5/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 28s 85ms/step - accuracy: 0.3930 - loss: 3.9521 - val_accuracy: 0.3939 - val_loss: 4.1408
Epoch 6/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - accuracy: 0.4027 - loss: 3.6912 - val_accuracy: 0.3782 - val_loss: 4.1099
Epoch 7/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - accuracy: 0.4177 - loss: 3.4114 - val_accuracy: 0.3812 - val_loss: 4.1050
Epoch 8/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - accuracy: 0.4315 - loss: 3.1976 - 

In [6]:
# ===== Mozilla – Cell 6: DBRNN baseline evaluation (Hit@K) =====

import numpy as np

def hit_at_k(probs, y_true, k):
    topk = np.argsort(-probs, axis=1)[:, :k]
    hits = (topk == y_true.reshape(-1, 1)).any(axis=1)
    return float(hits.mean())

# probabilities on Mozilla test set
probs_base = model.predict(X_test, batch_size=BATCH_SIZE)

TOPKS = [1, 3, 5, 7, 9]
hrs_dbrnn = []

print("=== DBRNN baseline (Mozilla, no graph) ===")
for k in TOPKS:
    hr = hit_at_k(probs_base, y_test, k)
    hrs_dbrnn.append(hr)
    print(f"Top-{k} Hit Ratio: {hr:.4f}")

avg_hr_dbrnn = float(np.mean(hrs_dbrnn))
print(f"Average Hit Ratio DBRNN : {avg_hr_dbrnn:.4f}")


90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step
=== DBRNN baseline (Mozilla, no graph) ===
Top-1 Hit Ratio: 0.3772
Top-3 Hit Ratio: 0.4657
Top-5 Hit Ratio: 0.4989
Top-7 Hit Ratio: 0.5254
Top-9 Hit Ratio: 0.5455
Average Hit Ratio DBRNN : 0.4825


In [7]:
# ===== Mozilla – Cell 7: developer vectors + adjacency matrix =====

from sklearn.metrics.pairwise import cosine_similarity

# 7.1 bug representations from "doc_repr" layer
repr_model = models.Model(
    inputs=model.input,
    outputs=model.get_layer("doc_repr").output
)

bug_repr_all = repr_model.predict(X_all, batch_size=BATCH_SIZE)
repr_dim = bug_repr_all.shape[1]
print("Bug repr shape:", bug_repr_all.shape)

# 7.2 aggregate to developer-level vectors (mean of their bugs)
dev_repr   = np.zeros((n_devs, repr_dim), dtype=np.float32)
dev_counts = np.zeros(n_devs, dtype=np.int32)

for i, d in enumerate(y_all):
    dev_repr[d] += bug_repr_all[i]
    dev_counts[d] += 1

nonzero = dev_counts > 0
dev_repr[nonzero] /= dev_counts[nonzero][:, None]

print("Developer repr shape:", dev_repr.shape)

# 7.3 cosine similarity graph + k-NN sparsification
K_NEIGHBORS = 20   # same as Eclipse / Chromium

sim = cosine_similarity(dev_repr)
np.fill_diagonal(sim, 0.0)

A = np.zeros_like(sim, dtype=np.float32)
for i in range(n_devs):
    idx = np.argsort(-sim[i])[:K_NEIGHBORS]
    A[i, idx] = sim[i, idx]

# symmetrise
A = np.maximum(A, A.T)

# 7.4 row-normalise + self-loops
row_sums = A.sum(axis=1, keepdims=True) + 1e-8
A_norm = A / row_sums

SELF_LOOP = 0.6
A_smooth = SELF_LOOP * np.eye(n_devs, dtype=np.float32) + (1.0 - SELF_LOOP) * A_norm

print("A_smooth shape:", A_smooth.shape)


448/448 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step
Bug repr shape: (114493, 256)
Developer repr shape: (1843, 256)
A_smooth shape: (1843, 1843)


In [8]:
# ===== Mozilla – Cell 8: GRCNN-style smoothing (developer graph) =====

STEPS = 2   # propagation steps

print("=== GRCNN-style (DBRNN + developer graph smoothing) on Mozilla ===")

probs_graph = probs_base.copy()
for _ in range(STEPS):
    probs_graph = np.matmul(probs_graph, A_smooth)

TOPKS = [1, 3, 5, 7, 9]
hrs_grcnn = []

for k in TOPKS:
    hr_g = hit_at_k(probs_graph, y_test, k)
    hrs_grcnn.append(hr_g)
    print(f"Top-{k} Hit Ratio (graph): {hr_g:.4f}")

avg_hr_graph = float(np.mean(hrs_grcnn))
print(f"Average Hit Ratio GRCNN : {avg_hr_graph:.4f}")

print("\nSummary on Mozilla:")
print(f"  DBRNN  avg HR : {avg_hr_dbrnn:.4f}")
print(f"  GRCNN* avg HR : {avg_hr_graph:.4f}  (GRCNN-style smoothing)")


=== GRCNN-style (DBRNN + developer graph smoothing) on Mozilla ===
Top-1 Hit Ratio (graph): 0.3764
Top-3 Hit Ratio (graph): 0.4491
Top-5 Hit Ratio (graph): 0.4700
Top-7 Hit Ratio (graph): 0.4826
Top-9 Hit Ratio (graph): 0.4917
Average Hit Ratio GRCNN : 0.4540

Summary on Mozilla:
  DBRNN  avg HR : 0.4825
  GRCNN* avg HR : 0.4540  (GRCNN-style smoothing)
